In [2]:
import numpy as np
import random
from sympy import symbols, Eq, sympify, lambdify

In [3]:
x, y, z, t = symbols('x y z t')

In [8]:
#constants
GENERATIONS = 10000
POPULATION_SIZE = 5000
MUTATION_RATE = 0.4
MUTATION_RANGE = 10000
PICK_NUMBER_FROM_PREV_GEN = 1000
PICK_NUMBER_FOR_PARENTS = 1000
NUMBER_OF_EQUATIONS = None
STAGNATION_LIMIT = 10
MAX_VALUE = 1e7
MIN_VALUE = -1e7

In [34]:
def initial_generation():
    return np.random.uniform(MIN_VALUE, MAX_VALUE, size=(POPULATION_SIZE, NUMBER_OF_EQUATIONS))


def parse_equation_string(equation_string):
    left_expression, right_value = equation_string.split("=")
    left_expr = sympify(left_expression.strip())
    right_expr = float(right_value.strip())
    return Eq(left_expr, right_expr)


def crossover(p1, p2):
    alpha = random.random()
    return alpha * p1 + (1 - alpha) * p2


def mutate(individual):
    if random.random() > MUTATION_RATE:
        return individual
    noise = np.random.normal(0, MUTATION_RANGE, size=NUMBER_OF_EQUATIONS)
    mutated = np.clip(individual + noise, MIN_VALUE, MAX_VALUE)
    return mutated


def select_parent(population, scores):
    indices = random.sample(range(PICK_NUMBER_FOR_PARENTS), 20)
    best_index = min(indices, key=lambda i: scores[i])
    return population[best_index]


def geneticAlgorithm(numberOfEquation, fitNessFunction, functions):
    population = initial_generation()
    stagnation_count = 0
    previous_best = None

    for generation in range(GENERATIONS):
        scores = np.array([fitNessFunction(index, functions)
                          for index in population])
        sorted_indices = np.argsort(scores)
        population = population[sorted_indices]
        scores = scores[sorted_indices]

        current_best = population[0]
        current_loss = scores[0]

        if current_loss <= 0.00001:
            print(f"🧬 Stopped after {generation + 1} generations")
            break
        if previous_best is not None and np.allclose(current_best, previous_best, rtol=1e-5, atol=1e-5):
            stagnation_count += 1
        else:
            stagnation_count = 0

        previous_best = current_best.copy()

        if stagnation_count >= STAGNATION_LIMIT:
            print(
                f"🛑 ReInitialize generations  after {generation + 1} ")
            population = initial_generation()
            continue

        next_generation = population[:PICK_NUMBER_FROM_PREV_GEN].tolist()

        while len(next_generation) < POPULATION_SIZE:
            p1 = select_parent(population, scores)
            p2 = select_parent(population, scores)
            child = crossover(p1, p2)
            child = mutate(child)
            next_generation.append(child)

        population = np.array(next_generation)
        if (numberOfEquation == 2):
            print(
                f"Gen {generation+1}: x={current_best[0]:.4f}, y={current_best[1]:.4f} Loss={current_loss:.8f}")
        elif (numberOfEquation == 3):
            print(
                f"Gen {generation+1}: x={current_best[0]:.4f}, y={current_best[1]:.4f}, z={current_best[2]:.4f}, Loss={current_loss:.8f}")
        elif (numberOfEquation == 4):
            print(
                f"Gen {generation+1}: x={current_best[0]:.4f}, y={current_best[1]:.4f}, z={current_best[2]:.4f}, t={current_best[3]:.4f} Loss={current_loss:.8f}")
    return population[0]

In [7]:
# Q1

NUMBER_OF_EQUATIONS = 2


def q1fitness(ind, funcs):
    x_val, y_val = ind
    total_error = 0
    try:
        for f in funcs:
            val = f(x_val, y_val)
            total_error += abs(val)
    except Exception:
        return float('inf')
    return total_error


def q1_create_functions(parsed_eqs):
    funcs = []
    for eq in parsed_eqs:
        f = lambdify((x, y), eq.lhs - eq.rhs, 'numpy')
        funcs.append(f)
    return funcs


input_equations = []
for i in range(NUMBER_OF_EQUATIONS):
    equation = input(f"Enter equation {i+1} (e.g., ax+by=c): ")
    input_equations.append(equation)

parsed_eqs = [parse_equation_string(eq) for eq in input_equations]
funcs = q1_create_functions(parsed_eqs)

answer = geneticAlgorithm(2, functions=funcs, fitNessFunction=q1fitness)
final_loss = q1fitness(answer, funcs)

print("\n✅ Best solution found:")
print(f"x = {answer[0]:.2f}")
print(f"y = {answer[1]:.2f}")
print(f"🔍 Total error: {final_loss:.2f}")

Gen 1: x=97180.3912, y=-80721.9206 Loss=130089.33248876
Gen 2: x=-28954.2272, y=12391.1552 Loss=70440.20506532
Gen 3: x=2333.6658, y=-1609.6747 Loss=3773.64780762
Gen 4: x=-3.6452, y=-5.5848 Loss=67.73500839
Gen 5: x=36.0487, y=-25.7597 Loss=48.62668990
Gen 6: x=0.6962, y=2.1189 Loss=1.67349812
Gen 7: x=1.6726, y=1.2341 Loss=0.51406342
Gen 8: x=1.9783, y=1.0234 Loss=0.03176284
Gen 9: x=1.9966, y=1.0023 Loss=0.00569601
Gen 10: x=1.9991, y=1.0006 Loss=0.00143068
Gen 11: x=2.0000, y=1.0000 Loss=0.00001752
🧬 Stopped after 12 generations

✅ Best solution found:
x = 2.00
y = 1.00
🔍 Total error: 0.00


In [35]:
# Q2
NUMBER_OF_EQUATIONS = 3


def q2fitness(ind, funcs):
    x_val, y_val, z_val = ind
    total_error = 0
    try:
        for f in funcs:
            val = f(x_val, y_val, z_val)
            total_error += abs(val)
    except Exception:
        return float('inf')
    return total_error


def q2_create_functions(parsed_eqs):
    funcs = []
    for eq in parsed_eqs:
        f = lambdify((x, y, z), eq.lhs - eq.rhs, 'numpy')
        funcs.append(f)
    return funcs


input_equations = []
for i in range(NUMBER_OF_EQUATIONS):
    equation = input(f"Enter equation {i+1} (e.g., 2*x + 3*y - z = 10): ")
    input_equations.append(equation)

parsed_eqs = [parse_equation_string(eq) for eq in input_equations]
funcs = q2_create_functions(parsed_eqs)

answer = geneticAlgorithm(3, functions=funcs, fitNessFunction=q2fitness)
final_loss = q2fitness(answer, funcs)

print("\n✅ Best solution found:")
print(f"x = {answer[0]:.2f}")
print(f"y = {answer[1]:.2f}")
print(f"z = {answer[2]:.2f}")
print(f"🔍 Total error: {final_loss:.3f}")

Gen 1: x=-185.7439, y=-7038981.4389, z=5672079.5870, Loss=8505851387.57374191
Gen 2: x=-100874.2610, y=-107410.4135, z=-19.6897, Loss=16333765.37395432
Gen 3: x=-73765.8742, y=-678134.1803, z=0.3070, Loss=2790184.03732070
Gen 4: x=-257.3772, y=-113406.8858, z=-146.9413, Loss=583332.41983180
Gen 5: x=-125.2264, y=13320.5026, z=-125.9856, Loss=187942.29186558
Gen 6: x=-2.1434, y=-3581.9225, z=289.0019, Loss=24976.42384858
Gen 7: x=-25.2466, y=196.8879, z=-1.5967, Loss=1388.49832036
Gen 8: x=2.8806, y=-23.9029, z=4.0500, Loss=172.90253527
Gen 9: x=-0.8827, y=-80.7602, z=-11.0879, Loss=72.57049067
Gen 10: x=1.3850, y=-5.6013, z=1.1511, Loss=21.30244834
Gen 11: x=0.9200, y=-4.0798, z=0.8983, Loss=4.80037061
Gen 12: x=0.7097, y=-5.3581, z=0.7998, Loss=2.06871029
Gen 13: x=0.6601, y=-4.9958, z=0.7370, Loss=0.35043031
Gen 14: x=0.6661, y=-5.0089, z=0.7502, Loss=0.02896573
Gen 15: x=0.6667, y=-4.9999, z=0.7505, Loss=0.01441402
Gen 16: x=0.6667, y=-4.9995, z=0.7500, Loss=0.00289386
Gen 17: x=0.6

In [17]:

# Q3
NUMBER_OF_EQUATIONS = 4


def q3fitness(ind, funcs):
    x_val, y_val, z_val, t_val = ind
    total_error = 0
    try:
        for f in funcs:
            val = f(x_val, y_val, z_val, t_val)
            total_error += abs(val)
    except Exception:

        return float('inf')
    return total_error


def q3_create_functions(parsed_eqs):
    funcs = []
    for eq in parsed_eqs:
        f = lambdify((x, y, z, t), eq.lhs - eq.rhs, 'numpy')
        funcs.append(f)
    return funcs


input_equations = []

for i in range(NUMBER_OF_EQUATIONS):
    equation = input(f"Enter equation {i+1} (e.g., 2*x + 3*y - z * t = 10): ")
    input_equations.append(equation)


parsed_eqs = [parse_equation_string(eq) for eq in input_equations]
funcs = q3_create_functions(parsed_eqs)
answer = geneticAlgorithm(4, functions=funcs, fitNessFunction=q3fitness)

final_loss = q3fitness(answer, funcs)
print("\n✅ Best solution found:")
print(f"x = {answer[0]:.2f}")
print(f"y = {answer[1]:.2f}")
print(f"z = {answer[2]:.2f}")
print(f"t = {answer[3]:.2f}")
print(f"🔍 Total error: {final_loss:.2f}")

Gen 1: x=404126.0714, y=2553427.7733, z=74855.8933, t=-8137514.1113 Loss=9679346.97450652
Gen 2: x=32121.3345, y=745189.7726, z=-61336.0838, t=-645121.5364 Loss=2870988.14821863
Gen 3: x=63951.3873, y=687104.6308, z=-744.4447, t=-1755106.6476 Loss=1102102.27804210
Gen 4: x=-10722.6390, y=-19531.2217, z=-5988.9031, t=53725.7179 Loss=304755.30068325
Gen 5: x=-554.4985, y=-20451.1647, z=-446.9640, t=14325.7091 Loss=88645.53447111
Gen 6: x=-355.6560, y=-1973.8068, z=337.5097, t=7213.7631 Loss=16051.04016822
Gen 7: x=118.5668, y=812.2722, z=-32.2002, t=-4071.0036 Loss=7962.18001394
Gen 8: x=30.9408, y=116.9659, z=-2.7024, t=-329.8174 Loss=551.74034859
Gen 9: x=-19.7667, y=-169.5966, z=5.2882, t=280.7646 Loss=316.22711467
Gen 10: x=-6.7293, y=-19.4323, z=-0.0376, t=48.7488 Loss=85.75310518
Gen 11: x=-1.5211, y=1.9447, z=0.0448, t=-5.1958 Loss=25.79274954
Gen 12: x=-1.6890, y=-8.0526, z=0.2088, t=6.6348 Loss=8.39220320
Gen 13: x=-1.5648, y=-4.9485, z=0.2617, t=0.8765 Loss=2.59108853
Gen 14: x